In [7]:
import cv2
import pdf2image
from PIL import Image
import numpy as np
from pytesseract import pytesseract
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import re

mpl.rcParams['image.cmap'] = 'gray'

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

In [8]:
start_pdf = 101
end_pdf = 101

In [9]:
def convert_pdf_to_image(file, dpi):
    images_list = []
    images = pdf2image.convert_from_path(file, dpi = dpi, poppler_path = r"C:\Users\Amandeep Singh\Downloads\Release-23.05.0-0\poppler-23.05.0\Library\bin")
    
    for i, image in enumerate(images):
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        images_list.append(image_np)
        
    return images_list

In [10]:
def f(x):
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(img).shape[1]//x

    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    
    cv2.imwrite("Images/verticle_lines.jpg",verticle_lines_img)
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

In [11]:
def sort_contours(cnts, method):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),key=lambda b:[b[1][1],b[1][0]] , reverse=False))
    return (cnts, boundingBoxes)

In [21]:
print("started")
for pdf_file in range(start_pdf, end_pdf+1):
    images = convert_pdf_to_image(str(pdf_file)+".pdf", 300)
    
    final_list = []
    for i in range(2, len(images)-1):
        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        
        (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_bin = 255 - img_bin
        cv2.imwrite("Images/Image_bin.jpg", img_bin)

        interact(f, x = widgets.IntSlider(min = 1, max = 255, step = 1, value = 50));
        
        alpha = 0.5
        beta = 1.0 - alpha
        img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
        img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
        (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        (contours, boundingBoxes) = sort_contours(contours,"top-to-bottom")
        
        g = []
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if(w < 800 and w > 500 and h > 280):
                g.append([x, y, w, h])
                
        a = []
        for i in range(len(g)):
            if i == 0:
                a.append(g[i])
            else:
                if np.abs(g[i][0] - g[i-1][0]) < 50:
                    pass
                else:
                    a.append(g[i])
                
        for i in a:
            new_img = img[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
            dic = {} 
            extracted_text = pytesseract.image_to_string(new_img, lang='ben', config=r'--oem 3 --psm 6')
            extracted_text = extracted_text.split()
            
            #Getting name from the list
            start_index_for_name = None
            end_index_for_name = None
            for i, item in enumerate(extracted_text):
                if item.startswith("নাম"):
                    start_index_for_name = i
                elif (item.startswith("স্বামী") or item.startswith("পিতা") or item.startswith("মায়ে") or item.startswith("অন্যা")) and end_index_for_name is None:
                    end_index_for_name = i
                    break
                    
            if start_index_for_name is not None and end_index_for_name is not None:
                name = ' '.join(extracted_text[start_index_for_name + 1:end_index_for_name])          
                name = name.strip()
                dic["Name"] = name
            else:
                dic["Name"] = "None"
                
            #Getting relation_name from the list
            start_index_for_relation = None
            end_index_for_relation = None
            relation = None
            for i, item in enumerate(extracted_text):
                if item.startswith("পিতা") and start_index_for_relation is None:
                    relation = "Father"
                    start_index_for_relation = i
                    break
                elif item.startswith("স্বামী") and start_index_for_relation is None:
                    relation = "Husband"
                    start_index_for_relation = i
                    break
                elif item.startswith("মায়ের") and start_index_for_relation is None:
                    relation = "Mother"
                    start_index_for_relation = i
                    
            for i, item in enumerate(extracted_text):
                if (item.startswith('বাড়ীর')) and end_index_for_relation is None:
                    end_index_for_relation = i
                    break
    
            if start_index_for_relation is not None and start_index_for_relation is not None:
                relation_name =  ' '.join(extracted_text[start_index_for_relation + 1:end_index_for_relation])
                
                patterns = r'নাম|\[|\]|\{|\}|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?|0|1|2|3|4|5|6|7|8|9'
                relation_name = re.sub(patterns, '', relation_name)
                relation_name = relation_name.strip()
                
            if relation == "Father":
                dic["FatherName"] = relation_name
            else:
                dic["FatherName"] = ""
                
            if relation == "Husband":
                dic["HusbandName"] = relation_name
            else:
                dic["HusbandName"] = ""
                
            if relation == "Mother":
                dic["MotherName"] = relation_name
            else:
                dic["MotherName"] = ""
            
            print(dic)
            print("==========")
            final_list.append(dic)
            
    sd = pd.DataFrame(final_list)
    sd[["Name", "FatherName", "HusbandName", "MotherName"]].to_excel(str(pdf_file) + ".xlsx")
    
    print(f"{pdf_file} completed")
    
print("===")
print("end")

started


interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': ': সোরেন তালামাই', 'FatherName': '', 'HusbandName': 'ধুমা মুর', 'MotherName': ''}
{'Name': ': আকতারি খাতুন', 'FatherName': '', 'HusbandName': 'গোলাম রসুল', 'MotherName': ''}
{'Name': ': আখতার আলম', 'FatherName': 'গোলাম রসুল', 'HusbandName': '', 'MotherName': ''}
{'Name': ': বিমল চন্দ্র সরকার', 'FatherName': 'প্রমথ নাথ সরকার ৩', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মানবেন্দ্র সরকার', 'FatherName': 'বিমল চন্দ্র সরকার ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সমরেন্দ্র সরকার', 'FatherName': 'বিমল চন্দ্র সরকার ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': শুক্লা সরকার', 'FatherName': 'বিমল চন্দ্র সরকার ও', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মৌমিতা সরকার (বিশ্বাস)', 'FatherName': '', 'HusbandName': 'কানাই লাল সরকার ৃ', 'MotherName': ''}
{'Name': ': রেখা সরকার', 'FatherName': '', 'HusbandName': 'বিমল সরকার', 'MotherName': ''}
{'Name': ': হেমরম পলুস', 'FatherName': 'রেনটা হেমরম ূ্\u200c', 'HusbandName': '', 'MotherName': ''}
{'Name': ': বরক

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': ': বিকাশ রায়', 'FatherName': 'অনন্ত রায় লিক ত*', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সাহাবুদ্দিন', 'FatherName': 'সমিম আখতার', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সাগ্তপ্তা খাতুন', 'FatherName': 'মোহম্মদ ইসলাম', 'HusbandName': '', 'MotherName': ''}
{'Name': ': নুর আলম', 'FatherName': 'মোহম্মদ সাবুর ূ্\u200c', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মোহম্মদ জামিল', 'FatherName': 'সুলেমান ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': রেখা সরকার', 'FatherName': '', 'HusbandName': 'ক্ষিতিশ সরকার ৪০০)', 'MotherName': ''}
{'Name': ': অজয় রায়', 'FatherName': 'অনন্ত কুমার রায় ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': শ্যামলী মন্ডল', 'FatherName': '', 'HusbandName': 'বিনয় কৃষ্ণ মন্ডল', 'MotherName': ''}
{'Name': ': রিজওয়ান আলম', 'FatherName': 'মোহম্মদ ইসলাম ৪ ০ + ॥', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সোমা সরকার', 'FatherName': '', 'HusbandName': 'মানবেন্দ্র সরকার নার', 'MotherName': ''}
{'Name': ': খিতিশ সরকার',

...

{'Name': ': মিলটন হালদার', 'FatherName': 'বিশ্বনাথ হালদার', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সুদাসী হালদার', 'FatherName': '', 'HusbandName': 'মনিমোহন হালদার', 'MotherName': ''}
{'Name': ': মনোজ কুমার হালদার', 'FatherName': 'মনিমোহন হালদার ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': ললিতা হালদার(সরকার)', 'FatherName': '', 'HusbandName': 'মনোজ কুমার হালদার ৃ', 'MotherName': ''}
{'Name': ': চৈতালি হালদার', 'FatherName': '-', 'HusbandName': '', 'MotherName': ''}
{'Name': ': বিশ্বনাথ হালদার', 'FatherName': 'নিত্যানন্দ হালদার ৃ  হ র', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মহঃ হাসিদ', 'FatherName': 'মহঃ সইরুদ্দিন | বয়স   লিঙ্গ পুং', 'HusbandName': '', 'MotherName': ''}


interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': ': বিক্রম বিশ্বাস', 'FatherName': 'বিমল বিশ্বাস নিল রঃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': অজিত দাস', 'FatherName': 'বীর মোহন দাস ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সুজিত দাস', 'FatherName': 'বীর মোহন দাস নী ১', 'HusbandName': '', 'MotherName': ''}
{'Name': ': প্রদীপ দাস', 'FatherName': 'দোমন দাস (১০০) ।!', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সুরোজিত রায়', 'FatherName': 'মন্মোঠো রয় রর ॥!', 'HusbandName': '', 'MotherName': ''}
{'Name': ': ষষ্টী বেন্দ্র বিশ্বাস', 'FatherName': 'বৈদ্যনাথ বিশ্বা', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সুকান্ত সিকদার', 'FatherName': 'সুশান্ত সিকদার ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মীনাক্ষী দাস', 'FatherName': '', 'HusbandName': 'অতেন দাস', 'MotherName': ''}
{'Name': ': রাজেশ চন্দ্র দাস', 'FatherName': 'অরুণ দাস ৪ পা', 'HusbandName': '', 'MotherName': ''}
{'Name': ': দীপজয় বিশ্বাস', 'FatherName': 'অশোক বিশ্বাস', 'HusbandName': '', 'MotherName': ''}
{'Name': ': জিতেন দাস'

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'Name': ': সরস্বতী মন্ডল', 'FatherName': 'জলাধর মন্ডল ৃ', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মনবর আলম', 'FatherName': 'মোহম্মদ সাবির আলম হি টু', 'HusbandName': '', 'MotherName': ''}
{'Name': ': সনবার আলম', 'FatherName': 'সাবির আলম রে ক', 'HusbandName': '', 'MotherName': ''}
{'Name': ': তালু টুড়ু', 'FatherName': 'দালাল টুড়ু (১০০) ।!', 'HusbandName': '', 'MotherName': ''}
{'Name': ': মালতী সরকার', 'FatherName': 'সুবল সরকার', 'HusbandName': '', 'MotherName': ''}
101 completed
===
end
